### Plot energy vs several detector hit properties
The values are provided from the csv file from the diretories ./output/files/{config}

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ROOT
from ROOT import gPad
import awkward as ak

Welcome to JupyROOT 6.26/06


In [2]:
# Variables
geometry = "showerMaxDetector_v3-1-2"
particle = "e"
particleName = "electron"
config = "qsim_61"
x_variable = "Beam energy [MeV]"
y_variable_res = " det resolution"
y_variable_pe = "Mean PE"
output_dir = "./output/plots/"

In [3]:
# Load the data
df = pd.read_csv(f"../output/files/{config}/eVsHitResults_{geometry}_{particle}.csv")
# Replae the nan values with 0
# Drop first row
df = df.drop([0])
df = df.replace("nan", 0)
df.keys()
df.head(20)



,Beam energy [MeV],Mean PE,mean PE error,rms,rms error,det resolution
1,2,0.0188,0.004121,0.412124,0.002914,21.9215
2,3,0.0130,0.004318,0.431777,0.003053,33.2136
3,4,0.0063,0.002237,0.223742,0.001582,35.5145
4,5,0.0056,0.001618,0.161767,0.001144,28.887
5,10,0.0251,0.002414,0.241392,0.001707,9.6172
6,15,0.0565,0.003882,0.388211,0.002745,6.87099
7,20,0.1053,0.005208,0.520780,0.003682,4.94568
8,25,0.1905,0.007571,0.757106,0.005354,3.97431
9,30,0.2793,0.009458,0.945776,0.006688,3.38624
10,35,0.3670,0.010473,1.047340,0.007406,2.85377


In [4]:
# Convert the data to float numpy
x = df[x_variable].to_numpy().astype(float)
y_pe = df[y_variable_pe].to_numpy().astype(float)
y_res = df[y_variable_res].to_numpy().astype(float)
print(x)

[2.0e+00 3.0e+00 4.0e+00 5.0e+00 1.0e+01 1.5e+01 2.0e+01 2.5e+01 3.0e+01
 3.5e+01 4.0e+01 4.5e+01 5.0e+01 5.5e+01 6.0e+01 6.5e+01 7.0e+01 7.5e+01
 8.0e+01 8.5e+01 9.0e+01 9.5e+01 1.0e+02 2.0e+02 3.0e+02 4.0e+02 5.0e+02
 6.0e+02 7.0e+02 8.0e+02 9.0e+02 1.0e+03 1.5e+03 2.0e+03 2.5e+03 3.0e+03
 3.5e+03 4.0e+03 4.5e+03 5.0e+03 5.5e+03 6.0e+03 6.5e+03 7.0e+03 7.5e+03]


In [5]:
%jsroot on
# Convert data to awkward array
x = ak.Array(x)
y_pe = ak.Array(y_pe)
y_res = ak.Array(y_res)
# print(x)
# print(y)
# print(type(x))

# Plot the data
c = ROOT.TCanvas("c", "c", 800, 600)
c.SetGrid()
gr = ROOT.TGraph(len(x))
for i in range(len(x)):
    gr.SetPoint(i, x[i], y_pe[i])
gr.SetMarkerStyle(20)
gr.SetMarkerSize(0.7)
gr.SetMarkerColor(ROOT.kBlue)
gr.SetTitle(f"Energy vs PE and Res for {geometry}:  Particle type: {particleName}")
gr.GetXaxis().SetTitle(x_variable)
gr.GetYaxis().SetTitle(y_variable_pe)
# gr.GetYaxis().SetRangeUser(0, 0.1)
# gr.GetXaxis().SetRangeUser(0, 5)
gr.Draw("AP")
c.Update()

# Now plot the resolution on same graph with different y axis to the right
gr_res = ROOT.TGraph(len(x))
for i in range(len(x)):
    gr_res.SetPoint(i, x[i], y_res[i])
gr_res.SetMarkerStyle(20)
gr_res.SetMarkerSize(0.7)
gr_res.SetMarkerColor(ROOT.kRed)

axis_right = ROOT.TGaxis(gPad.GetUxmax(), gPad.GetUymin(), gPad.GetUxmax(), gPad.GetUymax(), 0.15, 1.0, 510, "+L")
axis_right.SetTitle(y_variable_res)
axis_right.SetTitleOffset(1.2)
axis_right.SetTitleSize(0.03)
axis_right.SetLabelSize(0.03)
axis_right.Draw()

# Scale res graph to right axis
scale = gPad.GetUymax() / max(y_res)
#gr_res.Scale(scale)
gr_res.SetMarkerColor(ROOT.kRed)
gr_res.Draw("P same")

c.Draw()


In [14]:
# Plot PE vs Res
c2 = ROOT.TCanvas("c2", "c2", 800, 600)
c2.SetGrid()
gr_resVspe = ROOT.TGraph(len(x))
for i in range(len(x)):
    gr_resVspe.SetPoint(i, y_pe[i], y_res[i])
gr_resVspe.SetMarkerStyle(20)
gr_resVspe.SetMarkerSize(0.7)
gr_resVspe.SetMarkerColor(ROOT.kBlue)
gr_resVspe.SetTitle(f"PE vs Res for {geometry}:  Particle type: {particleName}")
gr_resVspe.GetXaxis().SetTitle(y_variable_pe)
gr_resVspe.GetYaxis().SetTitle(y_variable_res)

gr_resVspe.Draw("AP")
c2.Update()
c2.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2


In [6]:
# Save the plot
c.SaveAs(f"../output/plots/{config}/eVsPeRes_{geometry}_{particleName}.pdf")

Info in <TCanvas::Print>: pdf file ../output/plots/qsim_61/eVsPeRes_showerMaxDetector_v3-1-2_electron.pdf has been created


In [7]:
# Interpolate the data using TSpine
spline = ROOT.TSpline3("spline", gr)
spline.SetLineColor(ROOT.kRed)
spline.Draw("same")
#c.Draw()

# Predict the y values for given x values
print(spline.Eval(2345.67))


85.36599974992053


In [10]:
print(spline.Eval(1100))

40.756560503862275


In [16]:
# Create a root file to save TSpline for mean PE and RMS
f = ROOT.TFile(f"../output/files/{config}/tSpline_{geometry}_{particle}.root", "RECREATE")
spline_pe = ROOT.TSpline3("spline_peVsE", gr)
spline_res = ROOT.TSpline3("spline_resVsE", gr_res)
spline_resVspe = ROOT.TSpline3("spline_resVsPe", gr_resVspe)
spline_pe.Write()
spline_res.Write()
spline_resVspe.Write()

f.Close()


In [11]:
print(spline_pe.Eval(1100))
print(spline_res.Eval(1100))

# Get energy from PE


40.756560503862275
0.28820263131591495
